In [26]:
import networkx as nx
import pandas as pd
import matplotlib as mpl
import numpy as np
import matplotlib.pyplot as plt
import pygraphviz

In [27]:
A = pd.read_csv('data/ref_files/unordered_patterns_adjacency_matrix.csv', index_col=0, header=0)

In [28]:
G = nx.from_pandas_adjacency(A)

In [29]:
attrs = pd.read_csv('data/ref_files/unordered_patterns_base_attributes.tsv', sep='\t', header=0)
#pos_dict = {row['node']: (row['x'], row['y']*-1) for i, row in attrs.iterrows()}
layer = {row['node']: row['x'] for i, row in attrs.iterrows()}
#nx.set_node_attributes(G, pos_dict, "pos")
nx.set_node_attributes(G, layer, 'layer')

In [30]:
sx = 3
nx.set_node_attributes(G, nx.multipartite_layout(G, subset_key="layer", scale=3), "pos")
for n in G:
    G.nodes[n]['pos'] = "{},{}!".format(G.nodes[n]['pos'][0]*sx, G.nodes[n]['pos'][1])

In [31]:
def colorFader(c1,c2,mix=0): #fade (linear interpolate) from color c1 (at mix=0) to c2 (mix=1)
    c1=np.array(mpl.colors.to_rgb(c1))
    c2=np.array(mpl.colors.to_rgb(c2))
    return mpl.colors.to_hex((1-mix)*c1 + mix*c2)

In [32]:
# add proportion data to graph
proportion_file = pd.read_csv('etc/22_g-300_patterns.txt', sep='\t', header=0)

In [45]:
prop_labels = {row['node']: f'<{{<B>{row["node"][0]}U{row["node"][2]}R{row["node"][4]}C</B>|<FONT POINT-SIZE="10.0">{round(row["prop"],4)}</FONT>}}>' for i, row in proportion_file.iterrows()}
#prop_labels = {row['node']: '{{{}U{}R{}C | {}}}'.format(row["node"][0],row["node"][2], row["node"][4], row["prop"]) for i, row in proportion_file.iterrows()}
nx.set_node_attributes(G, prop_labels, 'label')

In [46]:
prop_colors = {row['node']: colorFader("white", "red", row['prop']) for i, row in proportion_file.iterrows()}
nx.set_node_attributes(G, prop_colors, 'fillcolor')

In [48]:
A = nx.nx_agraph.to_agraph(G)  # convert to a graphviz graph
A.layout(prog="neato", args="-n")
A.node_attr["shape"] = "record"
A.node_attr["style"] = "rounded,filled"
A.graph_attr["overlap"] ="scale"
A.draw("file.pdf", prog="neato", args="-n")  # Draw with pygraphviz
A.write("file.dot")